In [131]:
import numpy as np
import pandas as pd
import os

from sklearn.feature_selection import f_regression
from sklearn import metrics as sk_metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

import plotly.express as px
import plotly.graph_objects as go

In [14]:
DF_data

,EPA_PM2.5,time_hourly,PM2.5,Temperature,Humidity,HR
0,9.0,2019-09-01 08:00:00,3.105,33.808,50.316,8
1,8.0,2019-09-01 09:00:00,1.211,34.904,47.947,9
2,7.0,2019-09-01 10:00:00,1.650,34.901,48.700,10
3,4.0,2019-09-01 11:00:00,1.450,34.412,50.250,11
4,8.0,2019-09-01 16:00:00,1.632,30.406,59.211,16
...,...,...,...,...,...,...
4789,11.0,2020-03-31 19:00:00,17.389,22.803,87.611,19
4790,15.0,2020-03-31 20:00:00,21.053,22.879,87.474,20
4791,18.0,2020-03-31 21:00:00,24.400,22.798,88.500,21
4792,18.0,2020-03-31 22:00:00,42.316,22.011,91.947,22


In [263]:
JSON_ANA = {}
NEW_SITES = [ "banqiao_new", "taoyuan_new", "zhongming_new", "wanhua_new", "tainan_new", "qianjin_new" ]
for site in NEW_SITES:
    DF_data = pd.read_csv( "clean_data/mean/" + site + ".csv" )
    JSON_ANA[ site ] = {}
    JSON_ANA[ site ][ "covariance" ] = DF_data[["EPA_PM2.5", "PM2.5"]].cov()['EPA_PM2.5']['PM2.5']
    JSON_ANA[ site ][ "correlation" ] = DF_data[["EPA_PM2.5", "PM2.5"]].corr()['EPA_PM2.5']['PM2.5']
    JSON_ANA[ site ][ "error_rate" ] = (abs(DF_data["PM2.5"] - DF_data["EPA_PM2.5"]) / DF_data["EPA_PM2.5"] * 100).mean()
    #JSON_ANA[ site ][ "R2" ] = r2_score( DF_data["EPA_PM2.5"], DF_data["PM2.5"] )
    JSON_ANA[ site ][ "MSE" ] = mean_squared_error( DF_data["EPA_PM2.5"], DF_data["PM2.5"] )
    JSON_ANA[ site ][ "RMSE" ] = np.sqrt( JSON_ANA[ site ][ "MSE" ] )
    JSON_ANA[ site ][ "MAE" ] = mean_absolute_error( DF_data["EPA_PM2.5"], DF_data["PM2.5"] )
    JSON_ANA[ site ][ "R2" ] = np.corrcoef( x = DF_data[["EPA_PM2.5","PM2.5"]].T )[0][1] **2
    AirBox_Value( DF_data, site, "PM2.5", 10 )
    AirBox_Value( DF_data, site, "EPA_PM2.5", 6 )
    AirBox_Value( DF_data, site, "Temperature", 3 )
    AirBox_Value( DF_data, site, "Humidity", 3 )
    AirBox_Value( DF_data, site, "HR", 1 )

/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning:

invalid value encountered in true_divide

/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning:

invalid value encountered in true_divide



In [200]:
DF_ANA = pd.DataFrame(JSON_ANA)
DF_ANA.to_csv( "score_ana/score_all.csv" )

In [201]:
DF_ANA = pd.DataFrame(JSON_ANA)
DF_ANA

,banqiao_new,taoyuan_new,zhongming_new,wanhua_new,tainan_new,qianjin_new
covariance,115.874627,148.492719,219.080823,103.372517,181.719934,138.720902
correlation,0.841009,0.839426,0.892702,0.817846,0.783861,0.765282
error_rate,78.973729,90.667846,74.275925,85.960611,79.960722,70.097782
MSE,159.653615,232.051306,292.146599,174.025867,461.251483,337.962985
RMSE,12.635411,15.233230,17.092296,13.191886,21.476766,18.383770
MAE,10.279135,12.488545,14.219283,10.664978,17.748623,15.568895
R2,0.707296,0.704637,0.796916,0.668873,0.614438,0.585657


In [261]:
def AirBox_Value( DF_data, site, feature, p ):
    value = int(DF_data[feature].min())
    JSON_G = {}
    while value <= DF_data[feature].max() + p:
        value_mask = (value <= DF_data[feature]) & (DF_data[feature] < value + p)
        G = DF_data[ value_mask ]

        if(G.shape[0] > 1):
            JSON_G[ value ] = {}
            JSON_G[ value ][ "covariance" ] = G[["EPA_PM2.5", "PM2.5"]].cov()['EPA_PM2.5']['PM2.5']
            JSON_G[ value ][ "correlation" ] = G[["EPA_PM2.5", "PM2.5"]].corr()['EPA_PM2.5']['PM2.5']
            JSON_G[ value ][ "error_rate" ] = (abs(G["PM2.5"] - G["EPA_PM2.5"]) / G["EPA_PM2.5"] * 100).mean()
            JSON_G[ value ][ "R2" ] = np.corrcoef( x = G[["EPA_PM2.5","PM2.5"]].T )[0][1] **2
            JSON_G[ value ][ "MSE" ] = mean_squared_error( G["EPA_PM2.5"], G["PM2.5"] )
            JSON_G[ value ][ "RMSE" ] = np.sqrt( JSON_G[ value ][ "MSE" ] )
            JSON_G[ value ][ "MAE" ] = mean_absolute_error( G["EPA_PM2.5"], G["PM2.5"] )
        value += p
    
    # ouptut
    DF_AirBox_PM25 = pd.DataFrame(JSON_G)
    filepath = "score_ana/" + site + "/"
    filename = feature + ".csv"
    isExists=os.path.exists(filepath)
    if not isExists:
        os.makedirs(filepath)
    DF_AirBox_PM25.to_csv( filepath + filename )

In [264]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=DF_data.time_hourly, y=DF_data["PM2.5"],
                    mode='lines',
                    name='AirBox'))
fig.add_trace(go.Scatter(x=DF_data.time_hourly, y=DF_data["EPA_PM2.5"],
                    mode='lines',
                    name='EPA'))
fig.show()

In [265]:
DF_score = pd.read_csv( "score_ana/zhongming_new/Humidity.csv",index_col=0 )
DF_score

,32,35,38,41,44,47,50,53,56,59,...,71,74,77,80,83,86,89,92,95,98
covariance,133.930667,-0.992190,68.461662,81.609525,122.530787,140.951380,108.389397,124.302219,147.113670,215.193844,...,163.214646,187.311148,278.282115,263.581212,324.915658,296.190128,390.667156,547.032873,247.447803,16.227750
correlation,0.989765,-0.136433,0.806415,0.883953,0.883176,0.830506,0.824920,0.823260,0.875730,0.886650,...,0.914176,0.913008,0.947980,0.937606,0.929621,0.943185,0.948302,0.953732,0.899142,0.326556
error_rate,46.274603,46.199545,38.938389,44.945468,41.736384,49.798435,56.897140,54.620979,56.971877,53.272778,...,70.843980,83.213878,83.716954,82.930482,95.766335,115.850596,113.632792,115.248678,138.514077,215.697167
R2,0.979634,0.018614,0.650305,0.781373,0.779999,0.689741,0.680493,0.677758,0.766904,0.786148,...,0.835717,0.833583,0.898665,0.879106,0.864195,0.889598,0.899278,0.909604,0.808456,0.106639
MSE,53.242198,134.197424,81.801910,100.353780,88.768832,146.386279,138.041810,139.948909,149.592543,180.998247,...,222.940710,280.802349,324.841532,410.797728,552.324640,609.132641,621.370272,1111.388689,347.908042,312.442403
RMSE,7.296725,11.584361,9.044441,10.017673,9.421721,12.099020,11.749120,11.830000,12.230803,13.453559,...,14.931199,16.757158,18.023361,20.268146,23.501588,24.680613,24.927300,33.337497,18.652293,17.676040
MAE,5.935000,10.924857,7.796960,8.380167,7.500347,9.884090,9.950881,9.928982,10.268643,10.763813,...,13.233322,14.947140,16.070453,18.009150,20.905317,22.497690,20.144354,28.010328,13.326914,15.845200


In [266]:
fig2 = go.Figure()
Feature = "PM2.5"
# "PM2.5", "Temperature", "EPA_PM2.5", "HR", "Humidity"
Score = "R2"
# "correlation", "error_rate", "R2", "MSE", "RMSE", "MAE", "covariance"
for site in NEW_SITES:
    DF_score = pd.read_csv( "score_ana/"+ site +"/"+ Feature +".csv",index_col=0 )
    fig2.add_trace(go.Scatter(y=DF_score.loc[Score], x=DF_score.loc[Score].index,
                        mode='markers',
                        name=site))
fig2.show()